In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from tqdm.auto import tqdm


In [2]:
vacs = []
for i in tqdm(range(100)):
    url = f'https://my.sbertalents.ru/job-requisition/v2?page={i}&size=100'
    res = requests.get(url).json()
    if len(res['content']) > 0:
        vacs.extend(res['content'])
    else:
        break

In [3]:
def get_vacinfo(vhtml):
    vactext = ''
    vactitle = ''
    for k, v in vhtml.items():
        if k !='title':
            vactext += v
        else:
            vactitle = v
    soup = BeautifulSoup(vactext)
    vacdesc = soup.text.replace('\xa0',' ')
    return vactitle, vacdesc

In [4]:
vacs_dict = {
    'title':[],
    'description':[],
    'date':[],
    'id':[]
}

for vaca in tqdm(vacs):
    vtit, vdes = get_vacinfo(vaca['content'])
    vacs_dict['title'].append(vtit)
    vacs_dict['description'].append(vdes)
    vacs_dict['date'].append(vaca.get('date'))
    vacs_dict['id'].append(vaca['id'])

In [5]:
vtab = pd.DataFrame(vacs_dict)

In [6]:
vtab.head()

,title,description,date,id
0,Мобильный менеджер по продажам,Мы предлагаем:оформление согласно трудовому ко...,2020-11-24,395814
1,Специалист по прямым продажам,Мы предлагаем: трудоустройство согласно ТК РФ;...,2020-11-23,452801
2,Специалист по прямым продажам,Мы предлагаем: дистанционное обучение по банк...,2020-11-24,453517
3,Мобильный менеджер по продажам,Мы предлагаем: - оформление согласно трудовому...,2020-11-24,457243
4,Мобильный менеджер по продажам,Мы предлагаем:- оформление согласно трудовому ...,2020-11-24,460664


In [7]:
vtab[vtab.description.str.contains('python|Python')]

,title,description,date,id
37,Продуктовый аналитик,Условия:Конкурентоспособный уровень дохода (ок...,2020-11-23,1451548
39,Старший эксперт по технологиям,"Работа у нас — это:Команда профессионалов, гот...",2020-11-24,1457214
40,Старший инженер информационной безопасности банка,Мы предлагаем:работа в крупнейшем банке России...,2020-11-23,1459802
43,Старший инженер информационной безопасности,Условия: работа в крупнейшем банке России;труд...,2020-11-23,1462871
46,DevOps инженер,"Мы – большая, дружная команда, автоматизирующа...",2020-11-24,1462947
...,...,...,...,...
3073,Python (Голосовой ассистент),ЦЕЛИ И ЗАДАЧИ:Разработка серверной части высок...,2020-05-14,1163658
3074,Java разработчик,Machine Learning Engineer/Python разработчик К...,2020-05-13,1169089
3157,Аналитик Витрин Данных,Условия: работа в крупнейшем банке России;труд...,2020-01-13,777257
3171,Менеджер проектов,Ключевые навыки:Опыт проведения функциональног...,2019-12-18,999606


In [21]:
pat1 = 'python|Python'
pat2 = 'senior|Senior|руководитель|Руководитель|Главный|Директор'
t1 = vtab.description.str.contains(pat1)
t2 = vtab.title.str.contains(pat2)
# vtab[t1 & t2].title.tolist()

In [ ]:
data = []

In [9]:
smlist = []
litr = iter([row['id'] for _, row in vtab[t1 & t2].iterrows()])


In [10]:
lrows = [row['id'] for _, row in vtab[t1 & t2].iterrows()]

In [11]:
def previous_current_next(iterable):
    """Создает итератор который выдает таплы (предыдущий, текущий, следующий)
    
    Если нет значения, то значения предыдущего или следующего, то возвращает None

    """
    iterable=iter(iterable)
    prv = None
    cur = next(iterable)
    try:
        while True:
            nxt = next(iterable)
            yield (prv,cur,nxt)
            prv = cur
            cur = nxt
    except StopIteration:
        yield (prv,cur,None)

In [12]:
class VacNavigator:
    
    def __init__(self, vactuple):
        self.current_id = vactuple[1]
        self.previous_id = vactuple[0]
        self.next_id = vactuple[2]
        
    

In [13]:
vac_navis = [VacNavigator(tup) for tup in list(previous_current_next(lrows))]
vac_navis_dict = {v.current_id :v for v in vac_navis}

In [14]:
vac_navis_dict.get(1465482).next_id

1383390

In [15]:
def get_id_navigation(vac_id):
    return vac_navis_dict.get(vac_id)
     

In [16]:
vnav = get_id_navigation(1465482)
vnav.next_id

1383390

In [17]:
vnav.previous_id

In [18]:
import sys
sys.path.extend(['../'])
from dbsrc import DataAccessLayer, conn_string, Vacancy
conn_string

'postgresql://alex:@localhost:5432/sbtalents'

In [19]:
dal = DataAccessLayer(conn_string)
dal.connect()
# dal.session.
# dal.session.bulk_save_objects(vacancy_objects)
# dal.session.commit()

In [31]:
dal = DataAccessLayer(conn_string)
dal.connect()
list_of_vacs = [x[0] for x in dal.session.query(Vacancy.vacid).all()]
# list_of_vacs

In [49]:
list_of_vacs = dal.session.query(Vacancy.vacid).filter(Vacancy.vactitle.op('~')(r"python|Руководитель")).all()

[('1447405'),
 ('1467970'),
 ('1472781'),
 ('1459179'),
 ('1469139'),
 ('1464524'),
 ('1423294'),
 ('1443959'),
 ('1454533'),
 ('1468341'),
 ('1438765'),
 ('1442673'),
 ('819991'),
 ('1398688'),
 ('1434576'),
 ('1445342'),
 ('1444810'),
 ('1463002'),
 ('1452621'),
 ('1335777'),
 ('1335779'),
 ('1394825'),
 ('1458244'),
 ('1460731'),
 ('1464159'),
 ('766424'),
 ('1394826'),
 ('1435397'),
 ('1446553'),
 ('722915'),
 ('1423840'),
 ('1458974'),
 ('1434567'),
 ('1447566'),
 ('1170760'),
 ('1341572'),
 ('1346224'),
 ('1423838'),
 ('1453889'),
 ('1183483'),
 ('1334172'),
 ('1451788'),
 ('1455982'),
 ('1126596'),
 ('1415729'),
 ('1450451'),
 ('1452160'),
 ('1452892'),
 ('937624'),
 ('1401946'),
 ('1401945'),
 ('1451571'),
 ('1454625'),
 ('1363234'),
 ('1439022'),
 ('1439111'),
 ('1450120'),
 ('1450512'),
 ('1450069'),
 ('1450094'),
 ('1450154'),
 ('1431182'),
 ('1438717'),
 ('1447953'),
 ('827239'),
 ('1356884'),
 ('1164368'),
 ('1406690'),
 ('1434506'),
 ('1443807'),
 ('1444730'),
 ('1020461'

In [29]:
# def get_vacany_
def get_vacancy_obj(vac_id):
    dal = DataAccessLayer(conn_string)
    dal.connect()
    vobj = dal.session.query(Vacancy).filter(Vacancy.vacid == vac_id).first()
    return vobj


In [47]:
# # vac_id = list_of_vacs[3]
# for vac_id in list_of_vacs:
#     vob = get_vacancy_obj(vac_id)
#     print(vob.vactitle)

In [48]:
# vob.vactitle

In [ ]:
vobj.vactitle

In [106]:
class VacancyMessage:
    def __init__(self, vobj):
        self.vacdescription = vobj.vacdescription
        self.title  = vobj.vactitle
        self.vac_id = vobj.vacid
    
    def make_message(self):
        return (self.title, self.vacdescription, self.vac_id)
        

In [109]:
# vm = VacancyMessage(vobj)
# vm.make_message()

('Консультант по работе с ПФР',
 'Мы предлагаем:- трудоустройство согласно ТК РФ;- гарантированный доход плюс премиальное вознаграждение;- регулярное корпоративное обучение;- ДМС, страхование от несчастных случаев и тяжелых заболеваний;- материальную помощь и социальную поддержку, корпоративную пенсионную программу;- льготные условия кредитования;- яркую и насыщенную корпоративную жизнь.Как специалист, ты будешь:- обеспечивать выполнение плана по привлечению пенсионеров на получение пенсии через Сбербанк,- консультировать пенсионеров в закрепленном отделении Пенсионного фонда России по услуге получения пенсии через Сбербанк;- оформлять заявления /заявки на открытие счета карты Сбербанк-Maestro Социальная для пенсионера.  Ты подходишь нам, если ты:- имеешь среднее/среднее профессиональное/неполное высшее/высшее образование;- желателен опыт работы с клиентами;- владеешь стандартным ПО (Microsoft Excel, Outlook)на уровне базовых операций. Личные качества:- умение самостоятельно устанавлив

In [11]:
vn = get_id_navigation(1324943)

NameError: name 'get_id_navigation' is not defined

In [ ]:
vn.

In [18]:
vtitles = vtab[t1 & t2].title.tolist()

In [20]:
vdescs = vtab[t1 & t2].description.tolist()

In [21]:
vids = vtab[t1 & t2].id.tolist()

In [23]:
vdates = vtab[t1 & t2].date.tolist()

In [26]:
vacancies_dict = dict(zip(vids, tuple(zip(vtitles,vdescs,vdates))))

In [30]:
import pickle

In [32]:
with open('vac.pkl','wb') as f:
    pickle.dump(vacancies_dict,f)

In [33]:
with open('vac.pkl','rb') as f:
    vacancies_dict = pickle.load(f)

In [28]:

vacids = list(vacancies_dict.keys())

In [35]:
# vacancies_dict.get(vacids[0])

In [13]:
from dbsa import Vacancy, DataAccessLayer, conn_string

In [119]:
dal = DataAccessLayer(conn_string)
dal.connect()

In [121]:
query = dal.session.query(Vacancy).filter(Vacancy.vactitle.like('%Руководитель%'))

In [117]:
# for record in query:                       
#     print(record.vactitle)

In [122]:
query 